# Enviar job de Vertex
Enviar job de vertex para el entrenamiento del modelo en cloud

**This notebook has the same codes and comments showed in the previous folders of send jobs using pyton scripts. So, in this example all the comments were deleted**

In [ ]:
import datetime as dt
import pandas as pd

In [ ]:
from google.cloud import aiplatform
from google.cloud.aiplatform import gapic as aip

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv # package used in jupyter notebook to read the variables in file .env

""" get env variable from .env """
load_dotenv(find_dotenv())

""" Read env variables and save it as python variable """
PROJECT_ID_DS = os.environ.get("PROJECT_GCP", "")

### Paso 0. Parámetros generales

In [ ]:
### PARÁMETROS GENERALES GCP ###

# definir el proyecto. Diría que se tiene que hacer todo en un único proyecto porque sino vertex como que no funciona (creo)
PROJECT_ID = PROJECT_ID_DS

# definir un bucket (ya creado) para guardar los archivos que genera el usar VERTEX AI.
# No necesariamente puede ser un bucket, puede ser una carpeta dentro de un bucket. Lo cual es mejor si se quiere tener un
# bucket para un caso de uso y dentro de ese bucket un folder dedicado a todo los códigos de VERTEX AI
BUCKET_ID = '{bucket-name}/vertex-ai'

# definir una región, VERTEX SE INICIALIZA EN UNA REGIÓN y se utiliza algún recursos de vertex ej: datesets deben de estar 
# definidos en la misma región que se inicializó el código. Importante mayo 2023: dataset no están implementados para todas las regiones
REGION = '{region}'

In [ ]:
### PARÁMETROS GENERALES EJECUCIÓN ###

# obtener la hora actual de cuándo se comenzó la ejecución - hash
now = dt.datetime.now()
date_time = now.strftime("%Y_%m_%d_%H_%M_%S")


# identificacion del tipo de caso de uso (y también tipo de modelo) que se va a usar poara registrar el entrenamiento
identity_kind_use_case = 'job_vertex_registry_experiments'  


# definir path donde se va a guardar el pkl con el modelo y que además quede registrado en modelos de vertex
# definir path CUSTOM donde estará guardado el artefacto del modelo y el cual quedará registrado en el menu "modelos" de vertex
# obligatoriamente el path debe ser ".../model/model.pkl" por lo que la carpeta model y el artefacto model.pkl se omiten en este path
BUCKET_ID_MODEL = '{bucket-id-model}'
path_artifact_model_vertex = f'gs://{BUCKET_ID_MODEL}/poc-jobs-vertex/modeltypeA/run_{date_time}/'

In [ ]:
print('Parámetros Generales GCP')
print('PROJECT_ID: ', PROJECT_ID)
print('BUCKET_ID: ', BUCKET_ID)
print('REGION: ', REGION)

print('\n----')
print('Parámetros Específicos job entrenamiento')
print('date_time: ', date_time)
print('identity_kind_use_case: ', identity_kind_use_case)
print('path_artifact_model_vertex: ', path_artifact_model_vertex)

### Paso 1. Crear script de entrenamiento

In [ ]:
# ESCRIBIR EL SCRIPT DE ENTRENAMIENTO.

In [ ]:
# Correr script de entrenamiento de forma local - solo verificar
# %run train_model.py --id_date_time 2023_10_28_18_47_33

### Paso 2: Inicializar Vertex AI

In [ ]:
aiplatform.init(project = PROJECT_ID, location = REGION, staging_bucket = BUCKET_ID)

### Paso 3. Definir parámetros necesarios para CREAR la instancia del job de entrenamiento (aún no se envia)

In [ ]:
### definir el nombre del job que se enviará. Algo que indentifique de qué es el job + hora envio ###
job_name = identity_kind_use_case + '__job_train__' + date_time
job_name

In [ ]:
### definir el contrainer para el ENTRENAMIENTO y para LA PREDICCIÓN - facilitados por google ####
#container_train = 'us-docker.pkg.dev/vertex-ai/training/scikit-learn-cpu.0-23:latest' 

container_train = 'us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-12.py310:latest'
container_deploy = 'us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-23:latest' # solo utilizado para servicios de deploy y predicción batch de vertex

In [ ]:
### definir el path al script de entrenamiento ###
path_train_script = 'train_model.py'

In [ ]:
### definir la descripción del modelo ###
description = 'entrenar modelo leyendo pkl de GCS'

In [ ]:
### definir los requirements ###
# el db-types es necesario para poder consultar tablas de bigquery
list_requirements = ["google-cloud-bigquery==3.11.4", "db-dtypes", "gcsfs==2023.9.2", "pandas==2.0.3", "numpy==1.23.5", \
                     "scikit-learn==1.3.1", "google-cloud-aiplatform"]

### Paso 4. Definir parámetros necesarios para ENVIAR job de entrenamiento - usando CPU

In [ ]:
### definir el nombre con el que queda registrado (en VERTEX AI) el modelo resultado del entrenamiento ###
# De qué es el modelo +  hora de envio
model_name = identity_kind_use_case  + '__model__' + date_time 
model_name

In [ ]:
### definir el tipo de máquina para hacer el entrenamiento ###

machine_type_train = "n1-standard"
vcpu_train = "4"
train_compute = machine_type_train + "-" + vcpu_train

print("Train machine type: ", train_compute)

### Paso 5. Crear instancia del job de entrenamiento a VERTEX AI (CustomTrainingJob)

In [ ]:
# PRIMERO SE LLAMA UNA INSTANCIA DE LA CLASE
job = aiplatform.CustomTrainingJob(
    display_name = job_name,
    script_path = path_train_script,
    model_description = description,
    container_uri = container_train,
    requirements = list_requirements,
    model_serving_container_image_uri = container_deploy,
)

In [ ]:
job

### Paso 6. Enviar el job de entrenamiento a VERTEX AI (CustomTrainingJob)
**Add a personalized service account with the permission to registry in "vertex experiments"**

In [ ]:
# # # original

# model = job.run(
#     model_display_name = model_name,
#     replica_count = 1,
#     machine_type = train_compute,
#     base_output_dir = path_artifact_model_vertex, # path custom .../model/model.pkl donde se guarda el pkl del modelo. se omite del path model/model.pkl
#     args = ["--id_date_time=" + date_time], # args que se le pasan al script de entrenamiento de este ejemplo
#     sync = True
# )

In [ ]:
## ADD SA (service account)

model = job.run(
    model_display_name = model_name,
    replica_count = 1,
    machine_type = train_compute,
    base_output_dir = path_artifact_model_vertex, # path custom .../model/model.pkl donde se guarda el pkl del modelo. se omite del path model/model.pkl
    args = ["--id_date_time=" + date_time], # args que se le pasan al script de entrenamiento de este ejemplo
    sync = True,
    service_account = "SA"
)

In [ ]:
model